In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import time
import random
import os

In [2]:
def make_urllist_sub(index, date, page):
    sub_urllist = []
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(index)+'&date='+str(date)+'&page='+str(page)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)
    soup = BeautifulSoup(news.content, 'html.parser')
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
    for line in news_list:
        sub_urllist.append(line.a.get('href'))
    return sub_urllist

In [3]:
def make_urllist(index, date):
    total_url_list = []
    temp_url = []
    for page in range(1, 1000):
        url_list = make_urllist_sub(index, date, page)
        if url_list[-1] == temp_url:
            break
        temp_url = url_list[-1]
        total_url_list.extend(url_list)
    return total_url_list

In [5]:
def make_data(urllist, index, date):
    title_list = []
    text_list = []
    item_list = []
    for url in urllist:
        temp_sen = url.split('=')
        item_list.append(temp_sen[-1])
        article = Article(url, language='ko')
        article.download()
        article.parse()
        title_list.append(article.title)
        text_list.append(article.text)
    df = pd.DataFrame({'title': title_list, 'permalink': urllist, 'item_id': item_list, 'contents': text_list, 'section_id': index, 'datetime':date})
    return df

In [6]:
def make_total_data(index, startday):
    t = time.time()
    df = None
    temp_url = []
    endday = startday + 7  # 기본 설정은 1주일로 되어있습니다.
    for date in range(startday, endday):
        url_list = make_urllist(index, date)
        if temp_url == url_list[-1]:
            date = date -1
            break
        temp_url = url_list[-1]
        df_temp = make_data(url_list, index, date)
        #df_temp.to_csv(str(date) + '.csv') # 1일치가 끝날때마다 '해당 날짜.csv'로 저장됩니다. 필요하시면 활용하세요.
        if df is not None:
            df = pd.concat([df, df_temp])
        else:
            df = df_temp
        time.sleep(random.uniform(5, 10)) # 1일치가 끝나면 5초에서 10초동안 쉬도록 설정되어있습니다.
    df.to_csv(str(startday) + '-' + str(date) + '.csv') #일주일치가 끝날때마다 '첫날-마지막날.csv'로 저장됩니다.
    print(time.time()-t) # 1주일 치가 끝나면 해당 주차에 소요된 시간을 측정해줍니다.
    return df

In [7]:
def start(startday):
    for i in range(startday, startday + 31, 7):
        startday = i
        make_total_data(index, startday)
        time.sleep(random.uniform(30, 40))

## index

### 100 : 정치
### 101 : 경제
### 102 : 사회
### 103 : 생활/문화
### 104 : 세계
### 105 : IT/과학
### 110 : 오피니

# 파라미터는 index와 startday만 변경해주시면 됩니다.
# 더 필요하시면 make_total_data의 주석을 참고해주세요

In [8]:
index = 100 #인덱스 설정은 개인별로 바꾸시길 바랍니다.

In [9]:
startday = 20200901 # 작업 시작일입니다. 현재는 1일에서 시작하면 정상작동합니다. 자세한 내용은 노션 참조.

In [10]:
try :
    start(startday)
except  Exception :

    os.system('spd-say "Your code has trouble."') #문제가 생기면 이상한 기계음이 여러분을 부릅니다.
else :
    os.system('spd-say "Job done."') #작업이 완료되어도 부릅니다.

78.1746199131012
79.7487428188324
89.55101323127747
89.499098777771
25.190571784973145
